The goal here is to follow this tutorial 

https://docs.pymc.io/notebooks/GLM-rolling-regression.html

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import pymc3 as pm
import matplotlib.pyplot as plt

In [2]:
import os

In [3]:
GAMMA = 1/4

I didn't know how to do this in python, so I just exported this data frame from R. 
It is a "rolling window" of cumulative cases (with the window = 20).

You can find it here: https://github.com/tvladeck/covid-rt/blob/pymc3-approach/nyc-series.csv

In [4]:
nyc = pd.read_csv('~/Repos/covid-rt/nyc-series.csv')

In [5]:
nycp1 = nyc[2:len(nyc)]
nycm1 = nyc[1:(len(nyc)-1)]

In [6]:
model_randomwalk = pm.Model()

In [7]:
with model_randomwalk:
    # std of random walk
    sigma_theta = pm.HalfNormal('sigma_theta', sd = 0.1)

    theta = pm.GaussianRandomWalk('alpha', sd=sigma_theta,
                                  shape=len(nycp1.nyc))
    
    likelihood = pm.Poisson('y', mu = np.exp(theta) * nycm1.nyc, observed = nycp1.nyc)

In [ ]:
with model_randomwalk:
    trace_rw = pm.sample(tune=1000, cores=2,
                         target_accept=0.9)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [alpha, sigma_theta]
Sampling 2 chains:  62%|██████▏   | 1854/3000 [00:06<00:02, 504.12draws/s]

In [ ]:
rt = trace_rw['alpha'].T / GAMMA + 1
fig = plt.figure(figsize=(8, 6))
ax = plt.subplot(111, xlabel='time', ylabel='rt', title='Change of rt over time.')
ax.plot(rt, 'r', alpha=.05);